# Fake Review Detection Using BERT

In [ ]:
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

path = '/content/drive/MyDrive/SWM_Project/'

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/SWM_Project/

/content/drive/.shortcut-targets-by-id/1kN9YxqZQx6TWrX5kRlCxIDlOIDfX1KQg/SWM_Project


In [ ]:
# %cd drive/MyDrive/SWM_Project
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 4.0 MB 7.8 MB/s 
     |████████████████████████████████| 77 kB 7.9 MB/s 
     |████████████████████████████████| 6.6 MB 50.8 MB/s 
     |████████████████████████████████| 596 kB 59.1 MB/s 
     |████████████████████████████████| 895 kB 59.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 8.8 MB/s 


In [ ]:
# importing libraries
# import tokenization
from transformers import BertTokenizer
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import preprocessing
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score
import sqlite3
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from datetime import datetime
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from time import time
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, pairwise_distances
from sklearn.metrics import confusion_matrix
from collections import OrderedDict
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import json

pd.options.mode.chained_assignment = None

In [ ]:
%pwd

'/content/drive/.shortcut-targets-by-id/1kN9YxqZQx6TWrX5kRlCxIDlOIDfX1KQg/SWM_Project'

In [ ]:
# Loading the data

def load_data():
    print("Loading Data from Database")
    conn = sqlite3.connect("./yelpDataset.db")
    conn.text_factory = lambda x: str(x, 'gb2312', 'ignore')
    cursor = conn.cursor()

    # Create Review DataFrame
    cursor.execute(
        "SELECT reviewID, reviewerID, restaurantID, date, rating, usefulCount as reviewUsefulCount, reviewContent, flagged FROM review WHERE flagged in ('Y','N')")
    review_df = pd.DataFrame(cursor.fetchall(), columns=[column[0] for column in cursor.description])

    # Create Reviewer DataFrame
    cursor.execute("SELECT * FROM user")
    reviewer_df = pd.DataFrame(cursor.fetchall(), columns=[column[0] for column in cursor.description])

    # Create Restaurant DataFrame
    cursor.execute("SELECT restaurantID, rating as restaurantRating FROM business")
    restaurant_df = pd.DataFrame(cursor.fetchall(), columns=[column[0] for column in cursor.description])

    # Merge all DataFrames
    review_reviewer_df = review_df.merge(reviewer_df, on='reviewerID', how='inner')
    df = review_reviewer_df.merge(restaurant_df, on='restaurantID', how='inner')

    print("Data Load Complete")
    return df


df = load_data()
# df = df[:8000]
df.shape

# train_df = pd.DataFrame(train_data, train_label)
# test_df = pd.DataFrame(test_data, test_label)

Loading Data from Database
Data Load Complete


(23881, 21)

In [ ]:
df = df[:8000]

In [ ]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
train_df = train_data.copy()
test_df = test_data.copy()

In [ ]:
# Preprocessing the dataset
le = preprocessing.LabelEncoder()
train_df['label'] = le.fit_transform(train_df['flagged'])
train_df['input'] = train_df.apply(lambda x: '[CLS] '+ x['reviewContent'], axis = 1)
test_df['input'] = test_df.apply(lambda x: '[CLS] '+ x['reviewContent'], axis = 1)
train_texts = train_df.input.values.tolist()
train_labels = train_df.label.values.tolist()

In [ ]:
# Lavel Encoding
label = preprocessing.LabelEncoder()
y = label.fit_transform(train_df['flagged'])
y = to_categorical(y)
print(y[:5])

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [ ]:
# Downloading BERT Model
m_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(m_url, trainable=True)

In [ ]:
# Building Encoder
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
# tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
        
        text = text[:max_len-2]
        input_sequence = text
        pad_len = max_len-len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
        
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Designing Model Architecture
def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    
    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    
    clf_output = sequence_output[:, 0, :]
    
    lay = tf.keras.layers.Dense(128, activation='relu')(clf_output)
    lay = tf.keras.layers.Dropout(0.2)(lay)
    lay = tf.keras.layers.Dense(64, activation='relu')(lay)
    lay = tf.keras.layers.Dropout(0.2)(lay)
    out = tf.keras.layers.Dense(2, activation='softmax')(lay)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=2e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:

def Scores(y_true, pred, acc):
  test_acc = accuracy_score(y_true, pred)

  precision = precision_score(y_true, pred, average='weighted')

  recall = recall_score(y_true, pred, average='weighted')

  f1 = f1_score(y_true, pred, average='weighted')


  print("----------BERT SCORES----------")
  print("Train Accuracy: {0}".format(acc))
  print("Test Accuracy:  {0}".format(round(test_acc, 3)))
  print("Precision:      {0}".format(round(precision, 3)))
  print("Recall   :      {0}".format(round(recall, 3)))
  print("F1       :      {0}".format(round(f1, 3)))



In [ ]:
import numpy as np
max_len = 250

train_input = bert_encode(train_df.input.values, tokenizer, max_len=max_len)
test_input = bert_encode(test_df.input.values, tokenizer, max_len=max_len)
train_labels = y

In [ ]:
model1 = build_model(bert_layer, max_len=max_len)
model1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 250)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 250)]        0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 250)]        0           []                               
                                                                                                  
 keras_layer (KerasLayer)       [(None, 768),        109482241   ['input_word_ids[0][0]',         
                                 (None, 250, 768)]                'input_mask[0][0]',         

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# import torch

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device
# model1 = torch.cuda(model1)


In [ ]:

checkpoint = tf.keras.callbacks.ModelCheckpoint('fake_det_model_nd2.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)


# train_input = train_input.to(device)
# train_labels = train_labels.to(device)

train_sh = model1.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=10,
    callbacks=[checkpoint, earlystopping],
    batch_size=16,
    verbose=1
)

Epoch 1/10
320/320 [==============================] - ETA: 0s - loss: 0.5109 - accuracy: 0.7939
Epoch 1: val_accuracy improved from -inf to 0.81406, saving model to fake_det_model_nd2.h5
320/320 [==============================] - 350s 1s/step - loss: 0.5109 - accuracy: 0.7939 - val_loss: 0.4611 - val_accuracy: 0.8141
Epoch 2/10
320/320 [==============================] - ETA: 0s - loss: 0.4780 - accuracy: 0.7975
Epoch 2: val_accuracy did not improve from 0.81406
320/320 [==============================] - 303s 946ms/step - loss: 0.4780 - accuracy: 0.7975 - val_loss: 0.4351 - val_accuracy: 0.8141
Epoch 3/10
320/320 [==============================] - ETA: 0s - loss: 0.4264 - accuracy: 0.8107
Epoch 3: val_accuracy improved from 0.81406 to 0.81875, saving model to fake_det_model_nd2.h5
320/320 [==============================] - 334s 1s/step - loss: 0.4264 - accuracy: 0.8107 - val_loss: 0.4061 - val_accuracy: 0.8188
Epoch 4/10
320/320 [==============================] - ETA: 0s - loss: 0.3553 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-19-db72a5dfb87f>", line 15, in <module>
    verbose=1
  File "/usr/local/lib/python3.7/dist-packages/keras/utils/traceback_utils.py", line 64, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/engine/training.py", line 1389, in fit
    callbacks.on_train_batch_end(end_step, logs)
  File "/usr/local/lib/python3.7/dist-packages/keras/callbacks.py", line 438, in on_train_batch_end
    self._call_batch_hook(ModeKeys.TRAIN, 'end', batch, logs=logs)
  File "/usr/local/lib/python3.7/dist-packages/keras/callbacks.py", line 297, in _call_batch_hook
    self._call_batch_end_hook(mode, batch, logs)
  File "/usr/local/lib/python3.7/dist-packages/keras/callbacks.py", line 318, in _call_batch_end_hook
    self._call_batch_hook_helper(ho

KeyboardInterrupt: ignored

## Evaluating the BERT model trained on Fake Review Dataset

In [ ]:
preds = model1.predict(test_input)

In [ ]:
testdf_te = test_df.copy()
testdf_te['pred'] = ''
for i in range(len(testdf_te)):
  ls = list(preds[i])
  testdf_te['pred'].iloc[i] = ls.index(max(ls))

In [ ]:
testdf_te.to_csv('swm_bert_preds_10.csv', index=False)

In [ ]:
# Encoding 100 template labels to numeric form
label_dict = {}
for i,v in enumerate(train_df.flagged.unique().tolist()):
  label_dict[v] = i

testdf_te['y_true'] = testdf_te['flagged'].apply(lambda x: label_dict[x])

In [ ]:
# Evaluation Metrics

Scores(testdf_te['y_true'].values.tolist(), testdf_te['pred'].values.tolist(), '0.8335')
# testdf_te.columns

----------BERT SCORES----------
Train Accuracy: 0.8335
Test Accuracy:  0.789
Precision:      0.761
Recall   :      0.789
F1       :      0.771


In [ ]:
from sklearn.metrics import classification_report
# testdf_te['y_true'] = testdf_te['y_true'].apply(lambda x: int(x))
print('           ******** BERT Classification Report ********\n')
print(classification_report(testdf_te['y_true'].values.tolist(),testdf_te['pred'].values.tolist()))

           ******** BERT Classification Report ********

              precision    recall  f1-score   support

           0       0.84      0.91      0.87      1297
           1       0.41      0.27      0.33       303

    accuracy                           0.79      1600
   macro avg       0.63      0.59      0.60      1600
weighted avg       0.76      0.79      0.77      1600

